# Iniciemos sesion

In [4]:
TOKEN = 'putyourtokehereee'

# Primero las librerias a utilizar:

In [5]:
#Libraries for QC

import qiskit
import qiskit_aer
import qiskit_ibm_runtime


In [6]:
#general libraries:
import numpy as np

#Libraries for simulate molecules
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.hamiltonians import Hamiltonian, QuadraticHamiltonian
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper

from qiskit_nature.second_q.operators import FermionicOp

from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

from qiskit.circuit.library import EfficientSU2
from qiskit_algorithms import VQE, NumPyEigensolver

#Clasical minimizer
from scipy.optimize import minimize

#Plotting function
import matplotlib.pyplot

#poner mi token y mi sesion a futuro

In [7]:
LiH_molecule = 'H .0 .0 -{0}; Li .0 .0 {0}' #simulamos la molecula de Li-H primero
distances = np.arange(0.5, 4.25, 0.25) #diremos las distancias a las que estarán estas moleculas
vqe_energies = [] #energias obtenidas en cada iteracion
hf_energies = [] #energias iniciales del Anzats
exact_energies = [] #energias exactas


# Llamemos a nuestro simulador o CQ dependiendo de lo que necesitemos

In [8]:
from qiskit.primitives import StatevectorEstimator
from qiskit.providers.basic_provider import BasicSimulator
from qiskit_aer.primitives import Estimator

VQE_estimator = StatevectorEstimator()
VQE_estimator2 = Estimator()

In [80]:
#ocupar esto si se hacen simulaciones reales...

#from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

#target = backend.target


# Hagamos ahora la función de Callback:

In [9]:
callback_dict = {
    'prev_vector': None,
    'iters': 0,
    'cost_history': []
}


def build_callback(ansatz, hamiltonian, estimator, callback_dict):
    
    def callback(current_vector):
        callback_dict['iters'] += 1

        callback_dict['prev_vector'] = current_vector
        pub = (ansatz, [hamiltonian], [current_vector])
        result = estimator.run(pubs= [pub]).result()
        current_cost = result[0].data.evs[0]
        callback_dict['cost_history'].append(current_cost)

        print(
            f'Iters. done: {callback_dict['iters']} [Current cost: {current_cost}]'
        )
        
    return callback


# Función de Costo

In [10]:
def cost_func(params, ansatz, hamiltonian, estimator):

    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    energy = result[0].data.evs[0]

    return energy




# Preparemos el experimento:

In [27]:


import random


# Generamos nuestro Hamiltoniano

def get_qubit_op_LiH(dist, basis, map_type, verbose=False, tqr=False):

    #armamos la molecula
    LiH_molecule = 'H .0 .0 -{0}; Li .0 .0 {0}'

    #Hamiltoniano clasico:
    driver = PySCFDriver(LiH_molecule.format(dist/2),
                         basis= basis,
                         charge= 0,
                         spin= 0)
    
    qmol = driver.run()

    #calculamos las propiedades propias de la molecula
    repulsion_energy = qmol.nuclear_repulsion_energy
    num_particles = qmol.num_alpha + qmol.num_beta
    num_spin_orbitals = qmol.num_spin_orbitals * 2
    tuple = (qmol.num_alpha, qmol.num_beta)
    #calculamos las integrales de cuerpo (añadir tambien las de intercambio aqui)

    fermionic_op = qmol.hamiltonian.second_q_op()

    #hamiltonian = qmol.hamiltonian

    #print(hamiltonian)

    #h1 = hamiltonian.one_body_integrals
    #h2 = hamiltonian.twq_body_integrals

    #armamos el operador fermionico de nuestro hamiltoniano (2da cuantización)
    #ferOp = FermionicOp(h1= h1, h2=h2)

    #hacemos el mapeo del algebra de los operadores a y adagga a los de las matrices de pauli:
    #qubitOp = ferOp.mapping(map_type= map_type,
                            #threshold = 10 ** (-6))
    
    mapper = ParityMapper(num_particles= tuple)

    #proponer poner mas simetrias como la Z2 (tapered_mapper)
    qubitOp = mapper.map(fermionic_op)

    shift = repulsion_energy

    if verbose:
        #print(f'h1: {h1}')
        #print(f'h2: {h2}')
        print(qubitOp)
        print(qubitOp.print_details())

    if tqr:
        #qubitOp = buscar two qubit reduction
        pass
    
    print(tuple)
    return qubitOp, num_particles, num_spin_orbitals, shift, tuple
    pass


#Generamos nuestro Ansatz
def build_ansatz(qubitOp, num_orbitals, num_Particles, tuple, map_type='parity',
                 initial_state='HartreeFock', var_form= 'UCCSD', depth=1, tqr=False):

    mapper = ParityMapper(num_particles= tuple)
    print(f'num_orb: {num_orbitals}')
    print(f'num_pcle: {num_Particles}')

    initial_states = {
        'HartreeFock' : HartreeFock(num_spatial_orbitals= num_orbitals,
                                    num_particles= tuple,
                                    qubit_mapper= mapper)
    }
    pass

    #parametrizacion del circuito inicial
    var_forms = {
        'UCCSD' : UCCSD(num_spatial_orbitals= num_orbitals,
                        num_particles= tuple,
                        initial_state= initial_states[initial_state],
                        qubit_mapper= mapper,
                        reps= depth,
                        ),
        'EfficientSU2' : EfficientSU2(num_qubits= qubitOp.num_qubits,
                                      reps=depth,
                                      initial_state=initial_states[initial_state])
    }

    #initial_states[initial_state].draw('mpl')
   
    return var_forms[var_form]

#comparamos la respuesta Clasica
def exact_solver(qubitOp, verbose=False):
    ee = NumPyEigensolver(qubitOp)
    result = ee.run()
    ref = result['eigenvalues'].real[0]
    if verbose:
        print(f'Reference value: {ref}')
    return ref

#iniciamos en un punto random
def random_initial_point(ansatz, interval= [0, 1]):
    
    if len(interval) > 1:
        initial_point = [random(interval[0], interval[1]) for _ in range(ansatz.num_parameters)]
    
    else: 
        initial_point = [random(0, interval[0]) for _ in range(ansatz.num_parameters)]
    
    return initial_point


def random_init_point_V2(ansatz):
    #print(type(ansatz))
    #print(ansatz.num_parameters)



    return 2 * np.pi * np.random.random(ansatz.num_parameters_settable)




In [23]:
from qiskit_ibm_runtime import EstimatorV2
from qiskit_ibm_runtime.fake_provider import FakeKawasaki

fake_brisbane = FakeKawasaki()

final_estimator = EstimatorV2(backend= fake_brisbane)

In [35]:

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

#result_estimator = VQE_estimator.run( [  ] )

def callback1(nfev, parameters, energy, stddev):
    intermediate_info['nfev'].append(nfev)
    intermediate_info['parameters'].append(parameters)
    intermediate_info['energy'].append(energy)
    intermediate_info['stddev'].append(stddev)






for i, d in enumerate(distances):
    print('step', i)
    print(LiH_molecule.format(d/2))

    #primero armamos el problema físico
    qubitOp, num_pcle, num_spin_orb, shift, tup = get_qubit_op_LiH(
        dist= d, basis= 'sto-3g', map_type= 'parity'
    )
    
    #luego, armamos el Ansatz

    print(f'type QubitOp = {type(qubitOp)}')

    ansatz = build_ansatz(qubitOp= qubitOp, 
                          num_orbitals= num_spin_orb,
                          num_Particles= num_pcle,
                          map_type= 'parity',
                          tuple= tup,
                          initial_state='HartreeFock',
                          var_form= 'EfficientSU2',
                          depth= 2,
                          tqr= False)

    ansatz = EfficientSU2(num_qubits= qubitOp.num_qubits,
                          )

    target = fake_brisbane.target
    pm = generate_preset_pass_manager(target= target, optimization_level= 3)
    ansatz_isa = pm.run(ansatz)
    hamiltonian_isa = qubitOp.apply_layout(layout=ansatz_isa.layout)



    ini_point = random_init_point_V2(ansatz= ansatz)

    intermediate_info = {
        'nfev': [],
        'parameters': [],
        'energy': [],
        'stddev': []
    }
    
    from qiskit_algorithms.optimizers import COBYLA

    vqe = VQE(VQE_estimator2,
              ansatz_isa,
              COBYLA(maxiter=500),
              initial_point= ini_point,
              callback= callback1
              )

    
    min_energy = vqe.compute_minimum_eigenvalue(hamiltonian_isa)
    print(f'minimum_energy: {min_energy}')


    #vqe_result = np.real((result)['eigenvalue'] + shift)
    #callback = build_callback(ansatz_isa, hamiltonian_isa, VQE_estimator, callback_dict= callback_dict)
    
    #print(type(VQE_estimator))
    
    #res = minimize(
    #    cost_func,
    #    ini_point,
    #    args=(ansatz, qubitOp, VQE_estimator),
    #    method='cobyla',
    #    callback= callback,
    #    )
    






step 0
H .0 .0 -0.25; Li .0 .0 0.25
(2, 2)
type QubitOp = <class 'qiskit.quantum_info.operators.symplectic.sparse_pauli_op.SparsePauliOp'>
num_orb: 24
num_pcle: 4


AlgorithmError: 'The primitive job to evaluate the energy failed!'